# Module 5 Project

## Obtaining the datasets

In [1]:
import pandas as pd
df = pd.read_csv('train_users_2.csv')
df.head()

FileNotFoundError: [Errno 2] File b'train_users_2.csv' does not exist: b'train_users_2.csv'

In [ ]:
df.info()

### Obtaining 'sessions' dataset

In [ ]:
#sessions = pd.read_csv('sessions.csv')
#sessions.head()

## Data preparation and cleaning

### Redefining target

In [ ]:
#redefining target
#user who made a booking = 1
#user who did not make a booking = 0
df.loc[(df['country_destination'] != 'NDF','country_destination')] = 1
df.loc[(df['country_destination'] == 'NDF','country_destination')] = 0

In [ ]:
#renaming target column
df = df.rename(columns={'country_destination':'target'})

### Dropping unnecessary columns

In [ ]:
#drop date_first_booking - this column copies the target column and would cause data leakage
df.drop('date_first_booking',axis=1, inplace=True)

### Manipulating date_account_created and timestamp_first_active columns

In [ ]:
df.head()

In [ ]:
df['date_account_created']= pd.to_datetime(df['date_account_created']) 
df['timestamp_first_active']= pd.to_datetime(df['timestamp_first_active'])


In [ ]:
df.head() # timestamp_first_active shows incorrect dates, NEEDS FIXING!!

### Gender column

In [ ]:
df.gender.value_counts()

### Age column

In [ ]:
df.age.isna().sum()
#this column has 87990 missing values, that is more than a third of the dataset

#mean age before cleaning: 49.6
#median age before cleaning: 34


In [ ]:
df.age.value_counts()
#there are numerous errors in this column and many missing values
#for the sake of simplicity, remove all rows with errors and replace missing values with median age

#finding all ages over 110
indexNames = df[df['age'] > 110].index

#deleting these row indexes from dataFrame
df.drop(indexNames, inplace=True)

#replacing NaNs with median
df['age'].fillna((df['age'].median()), inplace=True)
#Note - replacing the NaNs with median did not affect the median of target users. This suggest that age is not
#defining element

df.age.isna().sum()

* Mean age after cleaning: 35 - significantly lower from 49 due to removing outliers
* Meadian age after cleaning: 34 - stayed the same

In [ ]:
df.signup_flow.value_counts()
#the page a user came to signup up from
#What does this mean?

In [ ]:
df.language.value_counts()
#since vast majority is english, this column might not bring much information.
#Also, it would create many caterogies and increase the dimensionality
#For now, I will drop whis column from the dataset

#UPDATE = I will keep this columns for experimentation
#df.drop('language', axis =1, inplace=True)

In [ ]:
df.columns

In [ ]:
df.affiliate_provider.value_counts()

In [ ]:
df.first_affiliate_tracked.isna().sum()
#whats the first marketing the user interacted with before the signing up
#6030 missing values

In [ ]:
df.first_affiliate_tracked.value_counts()

In [ ]:
#replacing NaNs with the most common value 'untracked"'
df.first_affiliate_tracked.fillna('untracked',inplace=True)

In [ ]:
df.first_browser.value_counts()
#many under-represented categories
#top 6 browsers = 209 509 users = 98.5% of users
#I will turn the remaining categories into 'other'
#UPDATE - For experiment I will keep this column as it is

#df.loc[(df.first_browser != 'Chrome') & (df.first_browser != 'Safari')& (df.first_browser != 'Firefox')& (df.first_browser != '-unknown-')& (df.first_browser != 'Mobile Safari')& (df.first_browser != 'IE'),'first_browser']='other'

### Preparing 'sessions' dataset (removed for insufficient data - sessions data only from the start of 2014 

In [ ]:
#sessions.head()

In [ ]:
#s_id = pd.Series(sessions.user_id.unique())
#df_id = pd.Series(df.id)

#print(len(s_id.unique()))
#print(len(df_id.unique()))
#each dataset has a different number of unique users

In [ ]:
#let's try merging the data to find whether we have any intersection
#s_id = s_id.rename('id')
#intersection = pd.merge(s_id, df_id, how='inner')
#len(intersection)
#it seems that we have 73 680 common users in each datasets

In [ ]:
#action_count = sessions.groupby(['user_id'])['action'].nunique() # number of unique actions a user performed on the website

In [ ]:
#Here I am mergin the unique numbers of actions per user with our main dataset
#action_count = pd.DataFrame(action_count)
#action_count = action_count.rename(columns={'0':'unique_sessions'})
#action_count = action_count.reset_index()
#action_count = action_count.rename(columns={'user_id':'id'})

In [ ]:
#the column 'action' will now contain a number of unique action a user has performed on the website
#df = pd.merge(df,action_count,how='inner')

In [ ]:
#df = df.drop('index',axis=1)

In [ ]:
df.head()

# EDA

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
#sns.set_style('whitegrid')


In [ ]:
plt.figure(figsize=(6,6))
plt.xlabel('Target', fontsize=15)
plt.ylabel('Frequency', fontsize=15)
plt.title('Number of users who did and did not book',fontsize=20)
plt.hist(df.target,color='dodgerblue',rwidth=5);

As we can see in this histogram, just over XXX users from the dataset did make a booking and around XXX did not. This suggests we have a target class imbalance which might have to be rectified by upsampling.

## Who are AirBnb customers?

Let's now have a look at the data from users who made a booking on the website and compare it with users who did not book. This might help us find indication of what are the important features. 

### AGE

In [ ]:
#creating separate datasets for users who did and did not book

#'yes booking = yb'
yb = df.loc[df.target == 1]

#'no booking = nb'
nb = df.loc[df.target == 0]

In [ ]:
plt.figure(figsize=(14,8))
plt.xlabel('Age', fontsize=15)
plt.ylabel('Frequency', fontsize=15)
plt.title('Age distribution',fontsize=20)
plt.hist(yb.age,color='darkorange',rwidth=5,alpha=1,label='Booked',bins=20)
plt.hist(nb.age,color='dodgerblue',rwidth=5,alpha=0.5,label='Did not book',bins=20)
plt.legend();

print('Median age of users who booked: '+str(round(yb.age.median(),2)))
print('Median age of users who did not book: '+str(round(nb.age.median(),2)))

* The age of users who did not book is more concentrated around the mean age (it has a strong peak in ages between 35 to 40).
* The peak of users who booked is in the previous bin, but the peak is less strong. 
* Most users are between 25 to 40. Both distributions have positive/right skew. 
* Median for both distributions is almost identical.


### GENDER

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x='gender',hue='target',data=df)
plt.xlabel('Gender', fontsize=15)
plt.ylabel('Frequency', fontsize=15)
plt.title('Gender',fontsize=20);

* The distribution of gender categories is pretty equal among users who made a booking, just below 10 000 each.
* Users whose gender is unknown were twice as likely not to make a booking compared to other gender categories.
* This seems to be a significant factor for determining a user who will not make a booking. 
* Also, if take into consideration the class imbalance mentioned above, users whose gender we know are generally more likely to make a booking than not!

### Sign-up method

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x='signup_method',hue='target',data=df)
plt.xlabel('Sign-up Method', fontsize=15)
plt.ylabel('Frequency', fontsize=15)
plt.title('Sign-up method',fontsize=20);

In [ ]:
df.head()

### When did users create their accounts

In [ ]:
dates_ac_created_yb = dict(yb.date_account_created.value_counts())
dates_ac_created_nb = dict(nb.date_account_created.value_counts())

In [ ]:
dict_ac_created_yb = dict()

for key in sorted(dates_ac_created_yb.keys()):
    dict_ac_created_yb.update({key:dates_ac_created_yb[key]})

dict_ac_created_nb = dict()

for key in sorted(dates_ac_created_nb.keys()):
    dict_ac_created_nb.update({key:dates_ac_created_nb[key]})

In [ ]:
series_ac_created_yb = pd.Series(dict_ac_created_yb)
series_ac_created_nb = pd.Series(dict_ac_created_nb)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(18,7), sharey=True)
axs[0].plot(series_ac_created_yb,color='darkorange')
axs[0].set_title('Users with booking',fontsize=15)
axs[1].plot(series_ac_created_nb,color='dodgerblue')
axs[1].set_title('Users without booking',fontsize=15)
fig.suptitle('Number of new accounts daily',fontsize=25);

### First browser

In [ ]:
plt.figure(figsize=(15,9))
sns.countplot(x='first_device_type',hue='target',data=df)
plt.xlabel('Device', fontsize=15)
plt.ylabel('Frequency', fontsize=15)
plt.title('Device',fontsize=20);

* By far the most popular devices as Mac and Windows desktop computers.
* Users of Mac Desktop are more likely to make a booking due to a small difference between users who booked it who did not


### Sign-up Flow (the page a user came to signup up from)

In [ ]:
plt.figure(figsize=(15,9))
sns.countplot(x='signup_flow',hue='target',data=df)
plt.xlabel('Source', fontsize=15)
plt.ylabel('Frequency', fontsize=15)
plt.title('Sign-up Flow - where users came from to sign up',fontsize=20);

* Unfortunately we don't know what sources there numbers represent, but most common by far is '0'
* Sources number 2 and 3 are most effective at converting users

In [ ]:
plt.figure(figsize=(15,9))
sns.countplot(x='affiliate_channel',hue='target',data=df)
plt.xlabel('Channel', fontsize=15)
plt.ylabel('Frequency', fontsize=15)
plt.title('Affiliate Channel - type of paid marketing',fontsize=20);

### Upsampling the minority class (0) to fix class imbalance

In [ ]:
from sklearn.utils import resample

# Separate majority and minority classes
#yb = class 1 = minority class
#nb = class 0 = majority class
 
# Upsample minority class
df_minority_upsampled = resample(yb,
                             	replace=True, 	# sample with replacement
                             	n_samples=len(nb),    # to match majority class
                             	random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
df= pd.concat([nb, df_minority_upsampled])
 
# Display new class counts
df.target.value_counts()


In [ ]:
plt.figure(figsize=(6,6))
plt.xlabel('Target', fontsize=15)
plt.ylabel('Frequency', fontsize=15)
plt.title('Number of users who did and did not book',fontsize=20)
plt.hist(df.target,color='dodgerblue',rwidth=5);

In [ ]:
df.head(20)

In [ ]:
#dropping unneccesary columns
df.drop('id',axis=1, inplace= True)

df.drop('date_account_created',axis=1, inplace= True)

df.drop('timestamp_first_active',axis=1, inplace= True)

### Creating dummy variables

In [ ]:
df = pd.get_dummies(data=df, columns=['gender', 'signup_method','signup_flow','language','affiliate_channel',
       'affiliate_provider', 'first_affiliate_tracked', 'signup_app',
       'first_device_type', 'first_browser'])

In [ ]:
len(df.columns)
df.head()
len(df)

 ### Splitting an initial testing set (10% of data)

In [ ]:
#I am going to cut away a random 10% of the dataset for final testing.

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

X_initial_split = df.drop('target',axis=1)
y_initial_split = df.target

#is = 'initial split'
X_train_is, X_test_is, y_train_is, y_test_is = train_test_split(X_initial_split, y_initial_split, test_size=0.1, random_state=123)

In [ ]:
len(X_train_is)
# this is 90% of our original dataset
# I will now overwrite this and will be using this to build all following models

In [ ]:
#Merging our dataset back together
df = pd.concat([X_train_is,y_train_is],axis=1)

## Fitting baseline simple model (random forest)

In [ ]:
#importing libraries
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score



In [ ]:
X = df.drop('target',axis=1)
y = df.target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)

In [ ]:
#importing libraries for PCA and scaling data

from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler


In [ ]:
#scaling data for PCA
scaler = StandardScaler()
scaled_X_train = pd.DataFrame(scaler.fit_transform(X_train))

In [ ]:
#fitting PCA components

pca_1 = PCA(n_components=110)
pca_2 = PCA(n_components=120)
pca_3 = PCA(n_components=130)

principalComponents = pca_1.fit_transform(scaled_X_train)
principalComponents = pca_2.fit_transform(scaled_X_train)
principalComponents = pca_3.fit_transform(scaled_X_train)

In [ ]:
#looking at explained variance ratio of different number of components
print(np.sum(pca_1.explained_variance_ratio_))
print(np.sum(pca_2.explained_variance_ratio_))
print(np.sum(pca_3.explained_variance_ratio_))

#120 components explains 97% of the variance

In [ ]:
#BASELINE MODEL WITHOUT GRIDSEARCH

pipe_baseline = Pipeline([('scl', MinMaxScaler()),
                  ('pca', PCA(n_components=120)),
                  ('clf', RandomForestClassifier(random_state = 123))])

pipe_baseline.fit(X_train,y_train)


In [ ]:
print(pipe_baseline.score(X_train,y_train))
print(pipe_baseline.score(X_test,y_test))

The baseline model successfully classified 69% of our testing data
Training accuracy was 77%, which suggests this model is overfitted.

In [ ]:
#DIAGNOsticks
from sklearn.metrics import classification_report



pipe_baseline_pred = pipe_baseline.predict(X_test)
print(classification_report(y_test, pipe_baseline_pred))
print(confusion_matrix(y_test, pipe_baseline_pred, labels=[1,2]))


In [ ]:
len(pipe_baseline_pred)

In [ ]:
#CONFUSION MATRIX

In [ ]:
from sklearn.metrics import roc_curve, auc
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, pipe_baseline.predict(X_test))
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

In [ ]:
#let's try this model with gridsearch
pipe_baseline_grid = Pipeline([('scl', MinMaxScaler()),
                  ('pca', PCA(n_components=120)),
                  ('clf', RandomForestClassifier(random_state = 123))])

# Set grid search params
param_baseline_grid = [ 
  {'clf__n_estimators': [30,50,80],
   'clf__criterion': ['gini','entropy'], 
   'clf__max_depth': [20,40],  
   'clf__min_samples_leaf':[0.02,0.002],  
  }]

# Construct grid search
gs_rf = GridSearchCV(estimator=pipe_baseline_grid,
            param_grid=param_baseline_grid,
            scoring='accuracy',
            cv=3, verbose=2, return_train_score = True)

# Fit using grid search
gs_rf.fit(X_train, y_train)

# Best accuracy
print('Best accuracy: %.3f' % gs_rf.best_score_)

# Best params
print('\nBest params:\n', gs_rf.best_params_)

# {'clf__criterion': 'gini', 'clf__max_depth': 20, 'clf__min_samples_leaf': 0.1, 'clf__n_estimators': 100}
# {'clf__criterion': 'gini', 'clf__max_depth': 20, 'clf__min_samples_leaf': 0.02, 'clf__n_estimators': 100}
#  {'clf__criterion': 'gini', 'clf__max_depth': 20, 'clf__min_samples_leaf': 0.002, 'clf__n_estimators': 100}

In [ ]:
#Best accuracy: 0.677

#Best params:
# {'clf__criterion': 'entropy', 'clf__max_depth': 40, 'clf__min_samples_leaf': 0.002, 'clf__n_estimators': 30}

In [ ]:
gs_rf_preds = gs_rf.predict(X_test)

In [ ]:
print(classification_report(y_test, gs_rf_preds))

In [ ]:
confusion_matrix(y_test,gs_rf_preds)

In [ ]:
#roc curve
fpr, tpr, thresholds = roc_curve(y_test, gs_rf_preds)
print('AUC: {}'.format(auc(fpr, tpr)))

In [ ]:
print('AUC: {}'.format(auc(fpr, tpr)))
plt.figure(figsize=(10, 8))
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve')
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.yticks([i/20.0 for i in range(21)])
plt.xticks([i/20.0 for i in range(21)])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

In [ ]:
#lets try a different model
from xgboost import XGBClassifier

In [ ]:
# Instantiate XGBClassifier
clf = XGBClassifier()

# Fit XGBClassifier
clf.fit(X_train, y_train)

# Predict on training and test sets
training_preds = clf.predict(X_train)
test_preds = clf.predict(X_test)

# Accuracy of training and test sets
training_accuracy = accuracy_score(y_train, training_preds)
test_accuracy = accuracy_score(y_test, test_preds)

print('Training Accuracy: {:.4}%'.format(training_accuracy * 100))
print('Validation accuracy: {:.4}%'.format(test_accuracy * 100))

In [ ]:
param_grid = {
    'learning_rate': [0.1, 0.2],
    'max_depth': [40],
    'min_child_weight': [1, 2],
    'subsample': [0.5, 0.7],
    'n_estimators': [30],
}



grid_clf = GridSearchCV(clf, param_grid, scoring='accuracy', cv=None, n_jobs=1)
grid_clf.fit(X_train, y_train)

best_parameters = grid_clf.best_params_

print('Grid Search found the following optimal parameters: ')
for param_name in sorted(best_parameters.keys()):
    print('%s: %r' % (param_name, best_parameters[param_name]))

training_preds = grid_clf.predict(X_train)
test_preds = grid_clf.predict(X_test)
training_accuracy = accuracy_score(y_train, training_preds)
test_accuracy = accuracy_score(y_test, test_preds)

print('')
print('Training Accuracy: {:.4}%'.format(training_accuracy * 100))
print('Validation accuracy: {:.4}%'.format(test_accuracy * 100))

In [ ]:
xboost_preds = grid_clf.predict(X_test)

In [ ]:
import stats_for_classification as stats

In [ ]:
stats.model_stats(y_test,xboost_preds,'XBOOST',False)